<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Build-Vocabulary" data-toc-modified-id="Build-Vocabulary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Build Vocabulary</a></span></li><li><span><a href="#Unigram-spell-checker" data-toc-modified-id="Unigram-spell-checker-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Unigram spell checker</a></span><ul class="toc-item"><li><span><a href="#Evaluate-accuracy" data-toc-modified-id="Evaluate-accuracy-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Evaluate accuracy</a></span></li></ul></li><li><span><a href="#Bigram-spell-checker" data-toc-modified-id="Bigram-spell-checker-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bigram spell checker</a></span><ul class="toc-item"><li><span><a href="#Evaluating" data-toc-modified-id="Evaluating-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Evaluating</a></span></li></ul></li></ul></div>

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from collections.abc import Iterable
import numpy as np
import nltk

import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
sys.path.insert(0, '../') 


import data
from data import utils

In [9]:
train, test = utils.get_evaluation()

In [10]:
train[0]

{'original': ['they', 'can', 'go', 'quite', 'farst'],
 'corrected': ['they', 'can', 'go', 'quite', 'fast'],
 'indexes': [4]}

In [11]:
X_tr = [x['original'] for x in train]
y_tr = [x['corrected'] for x in train]
y_tr_idx =  [x['indexes'] for x in train]

X_te = [x['original'] for x in test]
y_te = [x['corrected'] for x in test]
y_te_idx =  [x['indexes'] for x in test]

In [12]:
from nltk.collocations import BigramCollocationFinder


bigram_finder = BigramCollocationFinder.from_documents(X_tr)
bigram_freq_dict = dict(bigram_finder.ngram_fd.items())
list(bigram_freq_dict.keys())[0:10]

import itertools
vocabulary = set(list(itertools.chain(*bigram_freq_dict.keys())))
print(f'len(vocabulary)=|{len(vocabulary)}')

len(vocabulary)=|3451


In [13]:
bigram_freq_dict = dict(bigram_finder.ngram_fd.items())

# Build Vocabulary

In [14]:
import nltk
from nltk.collocations import ngrams

In [15]:
[x for x in nltk.collocations.ngrams(['the','man','went','to','the'],n=3)]

[('the', 'man', 'went'), ('man', 'went', 'to'), ('went', 'to', 'the')]

In [16]:
from collections import Counter

corrected_sentences = [train[n]['corrected'] for n in range(len(train))]
corrected_words = [word.lower() for sentence in corrected_sentences for word in sentence]
unique_corrected_words = set(corrected_words)
n_total_words = len(corrected_words)
vocabulary = unique_corrected_words

def build_unigrams(corrected_words):
    return Counter(corrected_words) 

def prob(word, unigrams, n_total_words):
    word = word.lower()
    word_counts = unigrams[word]
    return word_counts / n_total_words

# Test your code with the following
# assert(unigram("me")==87)

In [17]:
unigrams = build_unigrams(corrected_words)

In [18]:
print(prob('house', unigrams, n_total_words))
print(prob('me', unigrams, n_total_words))
print(prob('television', unigrams, n_total_words))

0.002200926223118896
0.003989178779402999
0.00018341051859324133


In [19]:
unigrams['house'], unigrams['me'], unigrams['television']

(48, 87, 4)

##### Get candidates

In [20]:
import nltk
from nltk import edit_distance

In [21]:

def get_candidates(token, vocabulary):
    # Write your code here.
    distance_token_to_words = {word:edit_distance(word,token.lower()) for word in vocabulary}
    minimum_distance = min(distance_token_to_words.values())
    return sorted([word for word, distance in distance_token_to_words.items() 
                   if distance <= minimum_distance], reverse=False)

In [22]:
# Test your code as follows
#assert get_candidates("minde",vocabulary) == ['mine', 'mind']

In [23]:
get_candidates("boxs",vocabulary) 

['boss', 'box', 'boys']

In [13]:
%timeit get_candidates("min",vocabulary)

34.6 ms ± 270 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Unigram spell checker

In [24]:
def correct_tokens(tokenized_sentence, unigrams, n_total_words):
    tokenized_sentence = tokenized_sentence.copy()
    
    for index,word in enumerate(tokenized_sentence):
        if (word and word.lower()) not in unique_corrected_words:
            candidates = {candidate:prob(candidate, unigrams, n_total_words) for candidate in get_candidates(word,vocabulary)}
            best_candidate  = max(candidates, key=candidates.get)
            tokenized_sentence[index] = best_candidate

    return tokenized_sentence

In [25]:
correct_tokens(tokenized_sentence = ["this", "whitr", "cat"],  
               unigrams=unigrams, 
               n_total_words=n_total_words)

['this', 'white', 'cat']

In [26]:
correct_tokens(tokenized_sentence = ["this","is","my","caat"],  
               unigrams=unigrams, 
               n_total_words=n_total_words)

['this', 'is', 'my', 'cat']

In [27]:
correct_tokens(tokenized_sentence = ["Joan","likes","pissa","and","cats"],  
               unigrams=unigrams, 
               n_total_words=n_total_words)

['Joan', 'likes', 'pigs', 'and', 'cats']

In [28]:
'pizza' in vocabulary

False

## Evaluate accuracy

In [29]:
def accuracy(test,  unigrams, n_total_words):
    # Write your code here
    count_total_words = 0
    count_corrected_words = 0
    for sentence in test:
        corrected_sentence = correct_tokens(sentence['original'], unigrams, n_total_words)  
        count_total_words +=len(sentence['corrected'])
        count_corrected_words += sum(corrected_sentence[n] == sentence['corrected'][n] 
                                     for n in range(len(sentence['corrected'])))
    return count_corrected_words/count_total_words

In [30]:
accuracy(test, unigrams, n_total_words)

0.8397887323943662

# Bigram spell checker

In [31]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics.distance import edit_distance
from collections import Counter

corrected_sentences = [train[n]['corrected'] for n in range(len(train))]
corrected_words = [word.lower() for sentence in corrected_sentences for word in sentence]
unique_corrected_words = set(corrected_words)
finder = BigramCollocationFinder.from_words(corrected_words)


In [32]:
corrected_words[0:10]

['they', 'can', 'go', 'quite', 'fast', 'this', 'was', 'a', 'royal', 'enfield']

In [33]:
unigram_freq_dict = build_unigrams(corrected_words)
bigram_freq_dict  = dict(finder.ngram_fd.items())

In [34]:
corrected_words[0:10]

['they', 'can', 'go', 'quite', 'fast', 'this', 'was', 'a', 'royal', 'enfield']

In [35]:
import itertools 
def top_k_dict(d, top_k = 10):
    return [(x,bigram_freq_dict[x]) for k,x in enumerate(bigram_freq_dict) if k<top_k]

top_k_dict(bigram_freq_dict)

[(('they', 'can'), 4),
 (('can', 'go'), 5),
 (('go', 'quite'), 1),
 (('quite', 'fast'), 1),
 (('fast', 'this'), 1),
 (('this', 'was'), 4),
 (('was', 'a'), 35),
 (('a', 'royal'), 3),
 (('royal', 'enfield'), 5),
 (('enfield', '_'), 1)]

Here we can notice that actually the bigrams where not computed correctly because we concatenated all the data from the  corrected sentences into a single list `corrected_words`

In [36]:
train[0]

{'original': ['they', 'can', 'go', 'quite', 'farst'],
 'corrected': ['they', 'can', 'go', 'quite', 'fast'],
 'indexes': [4]}

In [37]:
train[1]

{'original': ['this', 'was', 'a', 'Royl', 'Enfield', 'Consulatoin', '?', '_'],
 'corrected': ['this', 'was', 'a', 'Royal', 'Enfield', '_', '?', '_'],
 'indexes': [3, 5]}

Here we can see how the bigram `(fast,this)` appears in the `bigram_freq_dict` but in reality this might not even be in the training data.

It is a consequence of concatenating the training dat into a single line

In [38]:
list(bigram_freq_dict)[0:6]

[('they', 'can'),
 ('can', 'go'),
 ('go', 'quite'),
 ('quite', 'fast'),
 ('fast', 'this'),
 ('this', 'was')]

Note that we could build the bigrams avoiding this issue using `BigramCollocationFinder.from_documents`.

In [39]:
bigram_finder = BigramCollocationFinder.from_documents([['they', 'can', 'go', 'quite', 'fast'],
                                                         ['this', 'was', 'a', 'Royal', 'Enfield', '_', '?', '_']])
bigram_freq_dict_aux = bigram_finder.ngram_fd.items()
bigram_freq_dict_aux

dict_items([(('they', 'can'), 1), (('can', 'go'), 1), (('go', 'quite'), 1), (('quite', 'fast'), 1), (('this', 'was'), 1), (('was', 'a'), 1), (('a', 'Royal'), 1), (('Royal', 'Enfield'), 1), (('Enfield', '_'), 1), (('_', '?'), 1), (('?', '_'), 1)])

Let us build again `bigram_freq_dict_aux` using all the data

In [40]:
corrected_sentences = [train[n]['corrected'] for n in range(len(train))]
finder = BigramCollocationFinder.from_documents(corrected_sentences)
bigram_freq_dict = dict(finder.ngram_fd.items())

In [41]:
def prob_word(word, word_to_count, n_total_words, n_vocabulary):
    # Write your code here.
    word = word.lower()
    word_counts = word_to_count[word]
    return word_counts / n_total_words

#def prob_word(word, word_to_count, n_total_words, n_vocabulary): 
#    return (word_to_count[word]+1) / (n_total_words+ n_vocabulary)

def bigrams_starting_by(word, bigram_freq_dictionary): 
    return [t for t in list(bigram_freq_dict.keys()) if t[0] == word]

In [42]:
n_total_words, len(unique_corrected_words)
n_vocabulary = len(unique_corrected_words)

In [43]:
bigrams_starting_by('dog', bigram_freq_dict)

[('dog', 'see'),
 ('dog', 'and'),
 ('dog', 'run'),
 ('dog', 'is'),
 ('dog', ','),
 ('dog', 'had'),
 ('dog', 'for'),
 ('dog', 'Toby'),
 ('dog', '.'),
 ('dog', 'it')]

In [44]:
def return_dictionary_value(bigram, bigram_freq_dict):
    try:
        return bigram_freq_dict[bigram]
    except KeyError:
        return 0

def count_bigrams(list_bigrams, bigram_freq_dict): 
    return sum([return_dictionary_value(bigram, bigram_freq_dict) for bigram in list_bigrams])

In [45]:
count_bigrams([('they','can')], bigram_freq_dict)

4

In [46]:
len(bigram_freq_dict)

12218

In [47]:

def probability_bigram(word1, word2, bigram_freq_dict):
    if count_bigrams([(word1,word2)], bigram_freq_dict) == 0:
        return 0
    else:
        return count_bigrams([(word1,word2)], bigram_freq_dict)/count_bigrams(bigrams_starting_by(word1,bigram_freq_dict), bigram_freq_dict)


In [48]:
word1, word2  = ('dog','is')
probability_bigram(word1,word2, bigram_freq_dict)

0.09090909090909091

In [49]:
word1, word2  = ('was','a')
probability_bigram(word1,word2, bigram_freq_dict)

0.1263537906137184

In [50]:
word1, word2  = ('was','cat')
probability_bigram(word1, word2, bigram_freq_dict)

0

In [51]:
def interpolation_probability(word1, word2, bigram_freq_dict, n_vocabulary, lambda_1 = 0.3): 
    return (1-lambda_1)*probability_bigram(word1, word2, bigram_freq_dict) +\
            lambda_1*prob_word(word2, unigrams, n_total_words, n_vocabulary)


In [52]:
word1, word2  = ('was','a')
interpolation_probability(word1, word2, bigram_freq_dict, n_vocabulary)

0.09686619623303266

In [53]:
word1, word2  = ('was','cat')
interpolation_probability(word1, word2, bigram_freq_dict, n_vocabulary)

0.00020633683341739648

In [54]:
def get_candidates(token, vocabulary, max_dist):
    distance_token_to_words = {word:edit_distance(word,token.lower()) for word in vocabulary}
    minimum_distance = min(distance_token_to_words.values())
    if minimum_distance < max_dist:
        return sorted([word for word, distance in distance_token_to_words.items() if distance == minimum_distance])
    return [token]

In [55]:
get_candidates('huse', vocabulary, max_dist=2)

['house', 'huge', 'use']

In [56]:
get_candidates('kitch', vocabulary, max_dist=2)

['ditch', 'witch']

In [57]:
#%timeit get_candidates('hose', vocabulary, max_dist=2)

In [58]:

def correct_with_bigrams(sentence, vocabulary):
    for index,word in enumerate(sentence):
        if ((word and word.lower()) not in vocabulary) and (not word[0].isupper()):
            if index == 0: 
                previous_word = '.'
            else:
                previous_word = sentence[index-1].lower()
            candidates = {candidate:interpolation_probability(previous_word, candidate, bigram_freq_dict,
                                                              n_vocabulary, lambda_1=0.3) for candidate in get_candidates(word,vocabulary,max_dist=2)}
            
            
            sentence[index] = max(candidates, key=candidates.get)
    return sentence

In [59]:
correct_with_bigrams(["the","big","hose"], vocabulary)

['the', 'big', 'house']

In [60]:
word = 'hose'
previous_word = 'the'
candidates = {candidate:interpolation_probability(previous_word, candidate, bigram_freq_dict,n_vocabulary, lambda_1=0.3) for candidate in get_candidates(word,vocabulary,max_dist=2)}
candidates

{'hole': 0.0019202404016783775,
 'home': 0.0006602778669356688,
 'hope': 0.00012380210005043788,
 'horse': 1.37557888944931e-05,
 'hoses': 1.37557888944931e-05,
 'house': 0.00660255290938049,
 'lose': 4.12673666834793e-05,
 'nose': 4.12673666834793e-05,
 'rose': 6.87789444724655e-05,
 'those': 2.75115777889862e-05,
 'whose': 1.37557888944931e-05}

## Evaluating 


In [61]:
def accuracy_bigrams(test, vocabulary):
    # Write your code here
    count_total_words = 0
    count_corrected_words = 0
    mistakes = []
    for m,sentence in enumerate(test):
        s_true = sentence['corrected']
        s_hat  = correct_with_bigrams(sentence['original'].copy(), vocabulary)
        count_total_words  += len(s_true)
        correct_predictions = sum(s_hat[n] == s_true[n] for n in range(len(s_true)))
        count_corrected_words += correct_predictions
        if correct_predictions != len(s_true):
            mistakes.append(m)
            
    return count_corrected_words/count_total_words, mistakes

acc, mistakes = accuracy_bigrams(test, vocabulary)
acc

0.9014084507042254

In [62]:
i = 14
sentence = test[mistakes[i]]
x = sentence["original"]
y_true = sentence["corrected"]
y_hat  = correct_with_bigrams(sentence['original'].copy(), vocabulary)

print(f'mistake indices = {sentence["indexes"]}')
print(f'x      = {x}')
print(f'y_hat  = {y_hat}')
print(f'y_true = {y_true}')

mistake indices = [8, 12]
x      = ['1', 'night', 'when', 'it', 'was', 'dark', 'about', '12', 'oclock', 'a', 'man', 'was', 'nock', 'down', 'by', 'a', 'car', '.']
y_hat  = ['1', 'night', 'when', 'it', 'was', 'dark', 'about', '12', "o'clock", 'a', 'man', 'was', 'knock', 'down', 'by', 'a', 'car', '.']
y_true = ['1', 'night', 'when', 'it', 'was', 'dark', 'about', '12', "o'clock", 'a', 'man', 'was', 'knocked', 'down', 'by', 'a', 'car', '.']


In [63]:
i = 4
sentence = test[mistakes[i]]

x      = sentence["original"]
y_true = sentence["corrected"]
y_hat  = correct_with_bigrams(sentence['original'].copy(), vocabulary)

print(f'mistakes[i]={mistakes[i]}')
print(f'mistake indices = {sentence["indexes"]}')
print(f'x      = {x}')
print(f'y_hat  = {y_hat}')
print(f'y_true = {y_true}')

mistakes[i]=9
mistake indices = [9]
x      = ['I', 'go', 'to', 'bed', 'at', '10', 'o', 'clock', 'I', 'wakh', 'TV', 'at', '5', 'o', 'clock', 'I', 'live', 'in', 'a', 'house', '.']
y_hat  = ['I', 'go', 'to', 'bed', 'at', '10', 'o', 'block', 'I', 'wash', 'TV', 'at', '5', 'o', 'block', 'I', 'live', 'in', 'a', 'house', '.']
y_true = ['I', 'go', 'to', 'bed', 'at', '10', 'o', 'clock', 'I', 'watch', 'TV', 'at', '5', 'o', 'clock', 'I', 'live', 'in', 'a', 'house', '.']


In [64]:
test[8]

{'original': ['on', 'sundays', 'I', 'go', 'to', 'church', '.'],
 'corrected': ['on', 'sundays', 'I', 'go', 'to', 'church', '.'],
 'indexes': []}

In [65]:
i = 20
sentence = test[mistakes[i]]
x      = sentence["original"]
y_true = sentence["corrected"]
y_hat  = correct_with_bigrams(sentence['original'].copy(), vocabulary)


print(f'mistakes[i]={mistakes[i]}')
print(f'mistake indices = {sentence["indexes"]}')
print(f'x      = {x}')
print(f'y_hat  = {y_hat}')
print(f'y_true = {y_true}')

mistakes[i]=33
mistake indices = [15, 26]
x      = ['The', 'murder', 'man', 'has', 'a', 'black', 'beard', 'The', 'next', 'day', 'one', 'of', 'the', 'policemen', 'were', 'killd', 'the', 'next', 'day', 'they', 'found', 'the', 'car', 'over', 'the', 'Hill', 'the', 'was', 'the', 'man', 'near', 'it', 'he', 'was', 'dead', '.']
y_hat  = ['The', 'murder', 'man', 'has', 'a', 'black', 'heard', 'The', 'next', 'day', 'one', 'of', 'the', 'policemen', 'were', 'killed', 'the', 'next', 'day', 'they', 'found', 'the', 'car', 'over', 'the', 'Hill', 'the', 'was', 'the', 'man', 'near', 'it', 'he', 'was', 'dead', '.']
y_true = ['The', 'murder', 'man', 'has', 'a', 'black', 'beard', 'The', 'next', 'day', 'one', 'of', 'the', 'policemen', 'were', 'killed', 'the', 'next', 'day', 'they', 'found', 'the', 'car', 'over', 'the', 'Hill', 'there', 'was', 'the', 'man', 'near', 'it', 'he', 'was', 'dead', '.']
